In [40]:
# Import 
import gensim.downloader

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import _helpers as hp

# Load Data

In [41]:
ASR_data = pd.read_csv("snips/new_ASR_with_labels.csv") # ASR data with improved speech recognition with 15555 framerate and with autocorrection applied

GT_data = pd.read_csv("snips/merged_GT_data.csv") # Groundtruth data

## Data manipulation

In [42]:
#Get ASR data into a numpy word array per sentence plus a numpy label array

XX_ASR = ASR_data["transcript"].apply(lambda x: x.split(' '))
X_ASR = list(XX_ASR) #numpy word array per transcript

y_num_ASR = np.array(ASR_data["user_action_num"]) #labels

In [43]:
#Get Groundtruth data into a numpy word array per sentence plus a numpy label array

XX_GT = GT_data["transcript"].apply(lambda x: x.split(' '))
X_GT = list(XX_GT)  #numpy word array per transcript

y_num_GT = np.array(GT_data["user_action_num"]) #labels

# Word2Vec

## Import pre-trained Word2Vec model

In [44]:
model_w2v = gensim.downloader.load('glove-wiki-gigaword-100')

## Get features and labels

In [45]:
def get_Word2Vec_feature(sentence):
    """return word2vec numpy array representation of sentence"""
    
    value_iter = np.zeros((model_w2v.vector_size,))

    for word in sentence:
        try:
            word_vec = model_w2v[word]
            value_iter += np.array(word_vec)
        except:
            continue

    return value_iter

In [75]:
#features Space
features_ASR = np.stack(XX_ASR.apply(get_Word2Vec_feature).values, axis = 0)

#features Space
features_GT = np.stack(XX_GT.apply(get_Word2Vec_feature).values, axis= 0)

In [76]:
labels_ASR = ASR_data["user_action_num"]
labels_GT = GT_data["user_action_num"]

# Classifiers

## Train/Test Split

In [77]:
# Split into training and test data

train_features, test_features, train_labels, test_labels = train_test_split(features_ASR, labels_ASR, train_size= 0.9)

For train we will use both the training set of ASR data plus all the groudtruth data in order to have more data and build more robust models

In [78]:
train_features = np.concatenate((train_features,features_GT), axis=0)
train_features.shape

(3718, 100)

In [79]:
train_labels = np.concatenate((train_labels,labels_GT), axis=0)
train_labels.shape

(3718,)

## Logistic Regression

In [80]:
lgr = LogisticRegression(max_iter= 1000) #Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr) #Scale feature space
lgr_pipe.fit(train_features, train_labels)


lgr_pred_labels = lgr_pipe.predict(test_features) #predictions

lgr_score = lgr_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [81]:
print(classification_report(test_labels, lgr_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.79      0.79      0.79        24
     SwitchLightOn       0.90      0.93      0.92        29
IncreaseBrightness       0.88      0.76      0.82        38
DecreaseBrightness       0.80      0.75      0.77        16
SetLightBrightness       0.90      0.93      0.92        29
     SetLightColor       0.82      0.93      0.87        30

          accuracy                           0.86       166
         macro avg       0.85      0.85      0.85       166
      weighted avg       0.86      0.86      0.85       166

[[19  0  2  0  1  2]
 [ 1 27  1  0  0  0]
 [ 1  2 29  2  1  3]
 [ 3  0  1 12  0  0]
 [ 0  0  0  1 27  1]
 [ 0  1  0  0  1 28]]

ACCURACY: 0.8554216867469879


# Naive Bayes

### Gaussian Naive Bayes

In [82]:
gnb = GaussianNB() #Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb) #Scale feature space
gnb_pipe.fit(train_features, train_labels)


gnb_pred_labels = gnb_pipe.predict(test_features) #predictions

gnb_score = gnb_pipe.score(test_features,test_labels) #accuracy

#### Model Evaluation

In [83]:
print(classification_report(test_labels, gnb_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.59      0.67      0.63        24
     SwitchLightOn       0.79      0.79      0.79        29
IncreaseBrightness       0.67      0.32      0.43        38
DecreaseBrightness       0.28      0.31      0.29        16
SetLightBrightness       0.68      0.90      0.78        29
     SetLightColor       0.69      0.83      0.76        30

          accuracy                           0.64       166
         macro avg       0.62      0.64      0.61       166
      weighted avg       0.65      0.64      0.63       166

[[16  2  0  1  2  3]
 [ 1 23  1  1  0  3]
 [ 4  1 12 10  7  4]
 [ 4  2  3  5  1  1]
 [ 1  0  1  1 26  0]
 [ 1  1  1  0  2 25]]

ACCURACY: 0.6445783132530121


# SVM

In [84]:
svm = SVC() #Create the classification model

svm_pipe = make_pipeline(preprocessing.Normalizer(), svm) #Scale feature space
svm_pipe.fit(train_features, train_labels)


svm_pred_labels = svm_pipe.predict(test_features) #predictions

svm_score = svm_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [85]:
print(classification_report(test_labels, svm_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.72      0.75      0.73        24
     SwitchLightOn       0.82      0.93      0.87        29
IncreaseBrightness       0.93      0.66      0.77        38
DecreaseBrightness       0.61      0.69      0.65        16
SetLightBrightness       0.90      0.90      0.90        29
     SetLightColor       0.85      0.97      0.91        30

          accuracy                           0.82       166
         macro avg       0.80      0.81      0.80       166
      weighted avg       0.83      0.82      0.82       166

[[18  2  0  1  1  2]
 [ 0 27  1  0  1  0]
 [ 4  2 25  4  1  2]
 [ 2  2  1 11  0  0]
 [ 0  0  0  2 26  1]
 [ 1  0  0  0  0 29]]

ACCURACY: 0.8192771084337349


# NEURAL NETWORK

In [86]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100), activation='relu', solver='adam', max_iter=5000) #Create the classification model

mlp_pipe = make_pipeline(preprocessing.Normalizer(), mlp) #Scale feature space
mlp_pipe.fit(train_features, train_labels)


mlp_pred_labels = mlp_pipe.predict(test_features) #predictions

mlp_score = mlp_pipe.score(test_features,test_labels) #accuracy

### Model Evaluation

In [87]:
print(classification_report(test_labels, mlp_pred_labels, target_names= ['SwitchLightOff','SwitchLightOn','IncreaseBrightness','DecreaseBrightness','SetLightBrightness',"SetLightColor"]))

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.83      0.83      0.83        24
     SwitchLightOn       0.90      0.93      0.92        29
IncreaseBrightness       0.86      0.79      0.82        38
DecreaseBrightness       0.75      0.75      0.75        16
SetLightBrightness       0.92      0.83      0.87        29
     SetLightColor       0.83      0.97      0.89        30

          accuracy                           0.86       166
         macro avg       0.85      0.85      0.85       166
      weighted avg       0.86      0.86      0.85       166

[[20  0  0  1  1  2]
 [ 0 27  2  0  0  0]
 [ 2  2 30  1  0  3]
 [ 1  0  2 12  1  0]
 [ 1  1  1  1 24  1]
 [ 0  0  0  1  0 29]]

ACCURACY: 0.8554216867469879


# Try Your Self

In [114]:
def user_friendly(sentence, cls):
    """return action from sentence"""
    
    sent = sentence.split()
    new_sent = []

    for word in sent:
        new_sent.append(hp.autocorrection(word))
    
    x_pred = get_Word2Vec_feature(new_sent).reshape(1,-1)
    y_pred = cls.predict(x_pred)
    return hp.indx2action(y_pred)

In [116]:
user_friendly("I want to set brithenss to fifty in my living room", lgr_pipe)

['SetLightBrightness']